In [224]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

In [225]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [226]:
pip install scipy


In [227]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities


In [228]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [229]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [230]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [231]:
# cap = cv2.VideoCapture(0)
# # Set mediapipe model 
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():

#         # Read feed
#         ret, frame = cap.read()

#         # Make detections
#         image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
#         # Draw landmarks
#         draw_styled_landmarks(image, results)

#         # Show to screen
#         cv2.imshow('OpenCV Feed', image)

#         # Break gracefully
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()
    

In [232]:
# len(results.left_hand_landmarks.landmark)

In [233]:
# results

In [234]:
# draw_landmarks(frame, results)

In [235]:
# plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [236]:
#extreact key points

In [237]:
# results.pose_landmarks.landmark

In [238]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [239]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [240]:
face = np.zeros(1404)  # Default value in case no face landmarks are detected
if results.face_landmarks:
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten()


In [241]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [242]:
result_test = extract_keypoints(results)

In [243]:

result_test

array([ 0.54370755,  0.40232146, -1.51709318, ...,  0.        ,
        0.        ,  0.        ])

In [244]:
np.save('0', result_test)

In [245]:
np.load('0.npy')

array([ 0.54370755,  0.40232146, -1.51709318, ...,  0.        ,
        0.        ,  0.        ])

In [246]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['iam', 'good', 'boy'])

# Number of sequences per action
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start index
start_folder = 1

In [247]:
# os.makedirs(DATA_PATH, exist_ok=True)

# for action in actions:
#     action_path = os.path.join(DATA_PATH, action)

#     # Ensure the action directory exists before listing contents
#     os.makedirs(action_path, exist_ok=True)

#     # Get existing sequence folders and find the max index safely
#     existing_folders = []
#     if os.path.exists(action_path):  # Check if the directory exists
#         existing_folders = [int(folder) for folder in os.listdir(action_path) if folder.isdigit()]
    
#     dirmax = max(existing_folders) if existing_folders else 0  # Set to 0 if empty

#     # Create new sequence folders
#     for sequence in range(1, no_sequences + 1):
#         new_folder = os.path.join(action_path, str(dirmax + sequence))
#         os.makedirs(new_folder, exist_ok=True)  # Use exist_ok=True to avoid errors

In [248]:
# # Initialize Video Capture
# cap = cv2.VideoCapture(0)

# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

#     for action in actions:
#         for sequence in range(start_folder, start_folder + no_sequences):
#             for frame_num in range(sequence_length):

#                 # Read feed
#                 ret, frame = cap.read()

#                 # Check if frame is captured
#                 if not ret:
#                     print("⚠️ Warning: Unable to read frame. Skipping...")
#                     continue

#                 # Make detections
#                 image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
#                 results = holistic.process(image)

#                 # Convert back to BGR for OpenCV display
#                 image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#                 # Draw landmarks
#                 if results.face_landmarks:
#                     mp.solutions.drawing_utils.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
#                 if results.pose_landmarks:
#                     mp.solutions.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
#                 if results.left_hand_landmarks:
#                     mp.solutions.drawing_utils.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
#                 if results.right_hand_landmarks:
#                     mp.solutions.drawing_utils.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

#                 # NEW Apply wait logic
#                 if frame_num == 0:
#                     cv2.putText(image, 'STARTING COLLECTION', (120, 200),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
#                     cv2.putText(image, f'Collecting frames for {action} Video Number {sequence}', (15, 12),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     cv2.imshow('OpenCV Feed', image)
#                     cv2.waitKey(5000)
#                 else:
#                     cv2.putText(image, f'Collecting frames for {action} Video Number {sequence}', (15, 12),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     cv2.imshow('OpenCV Feed', image)

#                 # NEW Ensure directory exists before saving
#                 npy_folder = os.path.join(DATA_PATH, action, str(sequence))
#                 os.makedirs(npy_folder, exist_ok=True)  # ✅ Create missing directory
                
#                 # Extract keypoints function
#                 def extract_keypoints(results):
#                     pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
#                     face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
#                     lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
#                     rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
#                     return np.concatenate([pose, face, lh, rh])

#                 # Export keypoints
#                 keypoints = extract_keypoints(results)
#                 npy_path = os.path.join(npy_folder, f"{frame_num}.npy")
#                 np.save(npy_path, keypoints)  # ✅ Save keypoints

#                 # Break gracefully
#                 if cv2.waitKey(10) & 0xFF == ord('q'):
#                     break

# # Release webcam and close OpenCV windows
# cap.release()
# cv2.destroyAllWindows()

In [249]:
cap.release()
cv2.destroyAllWindows()

In [250]:
#pREPROCESS DATA AND CREATE LABELS AND FEATURES

In [251]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [252]:
label_map = {label:num for num, label in enumerate(actions)}

In [253]:
label_map

{'iam': 0, 'good': 1, 'boy': 2}

In [254]:
sequences, labels = [], []

for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):  # ✅ FIXED: Removed +1 to avoid extra frame
            npy_path = os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy")
            
            # ✅ FIX: Check if file exists before loading
            if os.path.exists(npy_path):
                res = np.load(npy_path)
                window.append(res)
            else:
                print(f"⚠️ Warning: Missing file {npy_path}, using zeros as placeholder.")
                window.append(np.zeros(1662))  # Adjust based on your keypoint size

        sequences.append(window)
        labels.append(label_map[action])  # Ensure label_map exists


In [255]:
# np.array(sequences).shape

In [256]:
# np.array(labels).shape

In [257]:
# X = np.array(sequences)

In [258]:
# X.shape

In [259]:
y = to_categorical(labels).astype(int)

In [260]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [261]:
y_test.shape

(5, 3)

In [262]:
#Build and train neural network

In [263]:
from tensorflow.keras.models import Sequential #sequential flow of neural network creation
from tensorflow.keras.layers import LSTM, Dense #a component to build the neural network
from tensorflow.keras.callbacks import TensorBoard #

In [264]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [265]:
# model = Sequential()
# model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
# model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(LSTM(64, return_sequences=False, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(actions.shape[0], activation='softmax'))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

# Define the model
model = Sequential([
    Input(shape=(30, 1662)),  # Define input shape here
    LSTM(64, return_sequences=True, activation='relu'),
    LSTM(128, return_sequences=True, activation='relu'),
    LSTM(64, return_sequences=False, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(actions.shape[0], activation='softmax')  # Ensure actions is defined
])



In [266]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=1800, callbacks=[tb_callback])

Epoch 1/1800
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - categorical_accuracy: 0.2716 - loss: 3.5981
Epoch 2/1800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - categorical_accuracy: 0.2993 - loss: 1.1749
Epoch 3/1800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - categorical_accuracy: 0.4174 - loss: 1.1074
Epoch 4/1800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - categorical_accuracy: 0.3624 - loss: 1.4111
Epoch 5/1800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - categorical_accuracy: 0.5272 - loss: 2.9389
Epoch 6/1800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - categorical_accuracy: 0.2917 - loss: 1.4594
Epoch 7/1800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - categorical_accuracy: 0.4235 - loss: 1.0979
Epoch 8/1800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - categorical_accuracy: 0.4568 - loss: 1.0947
Epoch 9/1800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - categorical_accuracy: 0.5197 - loss: 0.9744
Epoch 10/1800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - categorical_accuracy: 0.5218 - loss: 0.8784
Epoch 11/1800
3/3 ━━━━━━━━━━━

In [ ]:
# model.summary()

In [ ]:
#MAke predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
if len(res) > 4:
    predicted_action = actions[np.argmax(res[4])]
else:
    predicted_action = actions[np.argmax(res[0])]  

if len(y_test) > 4:
    actual_action = actions[np.argmax(y_test[4])]
else:
    actual_action = actions[np.argmax(y_test[0])]

print("Predicted Action:", predicted_action)
print("Actual Action:", actual_action)


In [ ]:
model.save('action.h5')

In [ ]:
# del model

In [ ]:
model.load_weights('action.h5')

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
# multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
# accuracy_score(ytrue, yhat)

In [ ]:
from scipy import stats

In [ ]:
# colors = [(245,117,16), (117,245,16), (16,117,245)]# Generate one color per action

# def prob_viz(res, actions, input_frame, colors):
#     output_frame = input_frame.copy()

#     # Debugging prints
#     print(f"res length: {len(res)}, actions length: {len(actions)}, colors length: {len(colors)}")

#     for num, prob in enumerate(res):
#         if num >= len(actions) or num >= len(colors):  # Prevent index error
#             print(f"Skipping index {num}, out of range")
#             continue
            
#         cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
#         cv2.putText(output_frame, actions[num], (0, 85 + num * 40), 
#                     cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

#     return output_frame

import cv2
import numpy as np
import matplotlib.pyplot as plt

colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]  # Generate one color per action

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()

    # Debugging prints
    print(f"res length: {len(res)}, actions length: {len(actions)}, colors length: {len(colors)}")
    print(f"res values: {res}")

    for num, prob in enumerate(res):
        if num >= len(actions) or num >= len(colors):  # Prevent index error
            print(f"Skipping index {num}, out of range")
            continue

        # Ensure prob is a scalar by taking the first value if it's an array
        if isinstance(prob, (np.ndarray, list)):
            prob = float(prob[0])  # Convert to a single float value
        
        # Draw probability visualization
        # cv2.rectangle(output_frame, (0, 60 + num * 40), (int(prob * 100), 90 + num * 40), colors[num], -1)
        # cv2.putText(output_frame, actions[num], (0, 85 + num * 40), 
        #             cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
# plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.85

cap = cv2.VideoCapture(0)

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            # 3. Viz logic
            if len(predictions) >= 10 and np.unique(predictions[-10:])[0] == np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
        
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
